In [3]:
import numpy as np
import pandas as pd
import yfinance as yf
import plotly.express as px

In [4]:
ticker = "IGSB"
data = yf.download(ticker, start="2007-01-01", end="2025-09-01", progress=False, auto_adjust=False)
data = data['Adj Close']
data.index.name = None
data.columns.name = None
data.columns = ['price']
data.to_csv(f"{ticker}.csv")

In [231]:
ticker = "ISFA.AS"
data = yf.download(ticker, start="2007-01-01", end="2025-09-01", progress=False, auto_adjust=False)
data = data['Adj Close']
data.index.name = None
data.columns.name = None
data.columns = ['price']
data.to_csv(f"{ticker}.csv")

In [245]:
df = pd.read_csv(f'{ticker}.csv', index_col=0, parse_dates=True)
df['price'] = df['price'].apply(lambda x: float(x))
df['change'] = df['price'].pct_change()
df = df.fillna(0)

df['rolling_change'] = df['change'].rolling(window=20).apply(lambda x: (1+x).prod()-1)
df = df.dropna()

df.loc[:, 'b'] = (df['rolling_change'] <= -0.01).astype(int).replace({1: np.nan, 0: 1}).shift().copy()

df.loc[:, 'ptf_change'] = df['b'] * df['change']

df['index'] = df['change'].add(1).cumprod()
df['ptf'] = df['ptf_change'].add(1).cumprod()

In [248]:
px.line(df['b'])

In [249]:
px.line(df[['index', 'ptf']].dropna())

In [250]:
# Calculate rolling maximum for both series
df['index_peak'] = df['index'].expanding().max()
df['ptf_peak'] = df['ptf'].expanding().max()

# Calculate drawdowns
df['index_drawdown'] = (df['index'] - df['index_peak']) / df['index_peak']
df['ptf_drawdown'] = (df['ptf'] - df['ptf_peak']) / df['ptf_peak']

# Plot drawdowns
fig = px.line(df[['index_drawdown', 'ptf_drawdown']],
              title='Drawdown Comparison',
              labels={'value': 'Drawdown', 'variable': 'Strategy'})
fig.show()

# Print maximum drawdowns
print(f"Maximum Index Drawdown: {df['index_drawdown'].min():.2%}")
print(f"Maximum Portfolio Drawdown: {df['ptf_drawdown'].min():.2%}")

Maximum Index Drawdown: -47.26%
Maximum Portfolio Drawdown: -45.24%


In [251]:
df[['change', 'ptf_change']].describe()

,change,ptf_change
count,"4,502.00000","2,995.00000"
mean,0.00021,-0.00005
std,0.01239,0.00985
min,-0.12341,-0.08657
25%,-0.00523,-0.00481
50%,0.00069,0.00045
75%,0.00607,0.00518
max,0.11041,0.05593


In [255]:
df['1y_rolling_change_index'] = df['change'].rolling(window=250).apply(lambda x: (1+x).prod()-1)
df['1y_rolling_change_ptf'] = df['ptf_change'].rolling(window=250).apply(lambda x: (1+x).prod()-1)

In [256]:
x = df['ptf_change'].dropna().rolling(window=250).apply(lambda x: (1+x).prod()-1)

In [257]:
px.line(x)

In [238]:
px.line(df[['1y_rolling_change_index', '1y_rolling_change_ptf']])

In [214]:
from python_module.pricing_model import BSMModel

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.options.display.float_format = '{:,.5f}'.format

In [262]:
strike = 70
premium = BSMModel.compute_option_with_forward(F=100, K=strike, T=1, r=0, sigma=0.3, option_type='put')-BSMModel.compute_option_with_forward(F=100, K=strike-1, T=1, r=0, sigma=0.3, option_type='put')

In [263]:
premium

0.14395031475664766